Excitation photons are emitted from the microscope via a titanium-sapphire laser in pulses at about 80 MHz, at a certain wavelength $\lambda=990$nm, and at a certain power, $I_0\approx 20$mW. This laser travels through roughly $b_0=350\mu$m of neural tissue to layer 2/3 of the neocortex where the beam is focused enough such that the probability of two photons from consecutive pulses exciting one fluorphor is high enough that it happens regularly. 

---

To model how much of the laser is absorbed in the neural tissue we will employ <a href="https://en.wikipedia.org/wiki/Beer%E2%80%93Lambert_law">Beer-Lambert's Law</a> wich determines how much light is transmitted from some material based on how much light as recieved by the material and some properties of the material. 

$$\Phi_t = \Phi_ie^{-\mu_a b c} \hspace{10pt} \text{where}$$ 

$\Phi_i = I_0$ is the amount of light arriving at one end of the material 

$\Phi_t$ is the amount of light being transmitted from the other end

$\mu_a$  is molar attenuation, the material's propensity to absorb light (determined below)

$b = b_0$ is the thickness of the sample, also called the "path length" of the material

$c$ is the concentration of the material in the sample that $\mu_a$ is defined for (determined below)

In trying to determine how much light neural tissue absorbs I came across <a href="http://www.ncbi.nlm.nih.gov/pubmed/23666068">this paper</a> which offers a means of approximating $\mu_a$. Otherwise we could just crudely approximate it by finding the most abundant molecules and assume they would dominate the absorption. For example, 62% of cerebrospinal fluid is composed of albumin, and all membranes are composed of lipid bilayers that all have very similar molar absorption coefficients. Most of this information and more can be found in <a href="http://bookzz.org/book/1128035/c6f178">this book</a>. Or we could neglect this all together. I just wanted to make sure I was aware of the assumptions being made.

---

Now, assuming we have determined the incident radiant energy reaching the indicator molecules, $\Phi_i$, we can again use <a href="https://en.wikipedia.org/wiki/Beer%E2%80%93Lambert_law">Beer-Lambert's Law</a> to approximate how much light is being emitted from the sample region within the point-spread function. Each fluophor has a particular quantum efficiency, $q$, which is the number of photons emitted divided by the number of photons absorbed. So we represent the amount of energy emitted as the product of the incident radiant energy and the quantum efficiency, $q\Phi_i$. However, the indicators also have a propensity to absorb some of the photons that were emitted by its neighbors and that must be accounted for as well, $q\Phi_ie^{-\mu_{a.ind}bc}$. Subtracting the second term from the first gives us an approximation to the fluorescence of a certain sample (or pixel in my case) as

$$F = q\Phi_i(1-e^{-\mu_{a.ind}bc}) \hspace{10pt} \text{where}$$

$F$ is the amount of light being emitted from the sample within the point spread function

$q = 0.610 \pm 0.004$ is the quantum efficiency of GCaMP6s, the indicator used in James' experiments

$\mu_{a.ind} = 2.91 \pm 0.04$ is the molar attenuation of GCaMP6s, the indicator used in James' experiments

$b \approx 3\mu$m is roughly the vertical variability of the point spread function (if I remember correctly)

$c$ is the concentration of indicator in the cell. I could not find a reference yet, but it is my understanding that this is assumed to be more or less equilibrated after some time and to be something that is independent of distance from the injection sight. This is because the indicator is delivered via a virus and once the cell has been infected it begins producing the proteins internally. While cells produce these proteins at different rates from the little bit I could find this variability should be small enough that a decently approximating constant value should exist. Or again, we can ignore this. I just want to make sure I know what I'm ignoring.

---

Now once the Fluorescence being emitted from the sample has been determined it must pass through the above neural tissue before it can be passed through the PMT and be recorded. So we can repeat the calculations done in the first part but with the emitted Fluorescence substituted in place of the incident randiance.

---

All that is left is modeling how the microscope transforms the analog fluorescence into a digital signal. I chose to ignore the quantum fluctions during sampling as that was beginning to go over my head. After reading over the microscope manual I believe everything else can be modeled. Starting with the PMT, it is essentially just adding noise so we model it by some function that returns the signal with added noise, $N(F)$. Then, as far as I'm aware, all that is left is to add the offset, $s\approx -1pA$ and multiplying by the gain g, which I couldn't find by examining the microscope. In addition, all of the parameters set for the microscope during each recording are saved in an xml file with the same name as the dataset. Therefore I have access to this data for each dataset if needed. Putting this together, the output fluorescence, $F_f$, that we see in our data would be

$$F_f = gN(F) + s$$

---

Now with all of the above information we can create some pretty ideal synthetic data. In addition, quite interestingly, we can also do this process in reverse and take a recorded measurement and actually determine the concentration of calcium ions that were present at the recording sight. And I don't mean approximate the number of calcium bound indicator, I mean the actual concentration of calcium that reacted with the indicator! (Thanks to knowing the $K_d$ for GCaMP6s).


Lets assume at some region bound by the point spread function there is a particular concentration of free Ca2+, $x$, and a particular concentration of free GCaMP, $y$, and a certain concentration of Ca2+ bound to the GCaMP, $z$.

$$[\text{Ca}2^+] = x+z$$
$$[\text{GCamP}] = y+z$$

The amount of Ca2+ bound to GCaMP is determined by its dissociation constant, $K_d$.

$$z = \frac{[\text{Ca}2^+] + [\text{GCaMP}]}{K_d + 2}$$

This would be for when I construct the synthetic data, however I can also use the $K_d$ to reverse the process and determine the $[\text{Ca}2^+]$ as long as we have one thing, the concentration of GCaMP...

$$z = \frac{\ln{\frac{F}{q\Phi_i}-1}}{bc}$$

Then...

$$[\text{Ca}2^+] = z(K_d+2)-[\text{GCaMP}]$$


**How about that!!**